In [ ]:
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!apt-get install -y fonts-nanum*

     |████████████████████████████████| 911kB 6.4MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted 

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from collections import deque

In [ ]:
# categories_sel = webdriver.find_elements_by_css_selector('#contents > section.reward-category > div')
# page_source = webdriver.page_source
# soup = bs(webdriver.page_source, 'html.parser')
# cate = soup.findAll('span', class_ = 'desc-category')
# categories = []
# for cat in cate:
#     categories.append(cat.text)


In [ ]:
# chrome_options = webdriver.ChromeOptions()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage') #, chrome_options=chrome_options)
chrome_options.add_argument('--start-maximized')

webdriver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
url = 'https://www.wadiz.kr/web/wsub/openfunding/reward?utm_source=rwm_gdn&utm_medium=search&utm_campaign=PC_%EB%A9%94%EC%9D%B4%EC%BB%A4&utm_content=PC_00.%EC%99%80%EB%94%94%EC%A6%88&utm_term=WADIZ&gclid=Cj0KCQjwyZmEBhCpARIsALIzmnKxj4JjlfxuvJXeYgAZOTzWwd2q72BLqExbdZtbLmHiiSbCxOOetM4aAn0kEALw_wcB'
webdriver.get(url)
body_main = webdriver.find_element_by_css_selector('body')

for i in range(5):
  body_main.send_keys(Keys.PAGE_DOWN)
  time.sleep(1)

webdriver.save_screenshot("website.png")

True

In [ ]:
# game = webdriver.find_element_by_css_selector('#contents > section.reward-category > div > ul > div > div > li:nth-child(14)')
# game.click()
# webdriver.save_screenshot("website.png")

ElementNotInteractableException: ignored

In [ ]:
# def crawling(category):

#     chrome_options = webdriver.ChromeOptions()
# driver = webdriver.Chrome('chromedriver')
# url = 'https://www.wadiz.kr/web/wsub/openfunding/reward?utm_source=rwm_gdn&utm_medium=search&utm_campaign=PC_%EB%A9%94%EC%9D%B4%EC%BB%A4&utm_content=PC_00.%EC%99%80%EB%94%94%EC%A6%88&utm_term=WADIZ&gclid=Cj0KCQjwyZmEBhCpARIsALIzmnJ4L-U5ZdU8R_6y2WpcNpXM6WVwglke6OpUP-fekXUudZSBTXWrlRAaAg0xEALw_wcB'

# driver.get(url)

#     # category 받아서 click()
#     game = webdriver.find_element_by_css_selector('#contents > section.reward-category > div > ul > div > div > li:nth-child(14) > a')
#     game.click()
last_height = webdriver.execute_script("return document.body.scrollHeight")
body = webdriver.find_element_by_tag_name("body")
while True:
    last_height = webdriver.execute_script("document.body.scrollHeight")
    scroll_down = 0
    while scroll_down < 10:

        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)
        webdriver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")

        time.sleep(0.3)

        scroll_down += 1

    new_height = webdriver.execute_script("document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height  

In [ ]:
#titles, categories, company_names, percentages, funding_amounts, days_left
titles, categories, company_names, percentages, funding_amounts, days_left = [],[],[],[],[],[]
contents = webdriver.find_elements_by_class_name('ProjectCardList_item__1owJa')
for content in contents:
    split = content.text.split('\n')
    if len(split) > 5:
        cont = deque(split)
        cont.popleft()
        title, category, company_name, target, day_left = list(cont)
    else:
        title, category, company_name, target, day_left = content.text.split('\n')
        
    titles.append(title)
    categories.append(category)
    company_names.append(company_name)
    percentages.append(target.split('%')[0])
    funding_amounts.append(target.split('%')[1])
    days_left.append(day_left)
len(titles)

0

In [ ]:
                  
#         하나씩 클릭해서 들어가야함 
page_source = webdriver.page_source
soup = bs(page_source, 'html.parser')
contents = soup.findAll('div',class_='ProjectCardList_item__1owJa')

n_supporters, likes = [],[]
durations, target_amounts = [],[]
descriptions = []

for i in range(1,len(titles)+1): #len(contents)
    btn = webdriver.find_element_by_css_selector(f'div.ProjectCardList_container__3Y14k > div > div:nth-child({i}) > div > div > a > div > span')
    time.sleep(0.5)
    btn.click()
    
    page_source = webdriver.page_source
    soup = bs(page_source, 'html.parser')
    
    #n_supporters, likes
    supporter = soup.find('p',class_ = 'total-supporter').text.split('명')[0]
    n_supporters.append(supporter)
    like = soup.find('em',class_='cnt-like').text
    likes.append(like.strip())

    #durations,target_amounts
    info = soup.find('p',style='color:#00b2b2;font-size:13px;line-height:20px;margin-bottom:8px;').text.split('\n')
    duration = info[2].split(' ')[-1]
    durations.append(duration)
    target_amount = info[1].split(' ')[-1]
    target_amounts.append(target_amount)

    #descriptions
    description = soup.select("#container > div.reward-body-wrap > div > div.wd-ui-info-wrap > div.wd-ui-sub-campaign-info-container > div > div > section > div.campaign-summary")[0]
    descriptions.append(description.text.strip())

    time.sleep(0.5)
    webdriver.back()
    
    
    


# 데이터 프레임 

In [ ]:
data = {
'title':titles,
'categories':categories,
"company_names":company_names,
'percentages':percentages, 
'funding_amounts' :funding_amounts, 
'days_left' :days_left
}    
df = pd.DataFrame(data)
df['percentages'] = percentages
df['funding_amounts'] = funding_amounts
df['n_supporters'] = n_supporters
df['likes'] = likes
df['descriptions'] = descriptions
df['durations'] = durations
df['target_amounts'] = target_amounts

#경로 추가가 필요할 수 있음
df.to_csv(category+'_df')


# 시계열 데이터 크롤링


In [ ]:
#이거 먼저 실행 되어야함이거 먼저 실행 되어야함
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage') #, chrome_options=chrome_options)
chrome_options.add_argument('--start-maximized')
webdriver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

In [ ]:
def timeseries_data(category):
        
    categories_sel = webdriver.find_elements_by_css_selector('#contents > section.reward-category > div')
    page_source = webdriver.page_source
    soup = bs(webdriver.page_source, 'html.parser')
    cate = soup.findAll('span', class_ = 'desc-category')
    categories = [cat.text for cat in cate]
    
    url = 'https://www.wadiz.kr/web/wsub/openfunding/reward?utm_source=rwm_gdn&utm_medium=search&utm_campaign=PC_%EB%A9%94%EC%9D%B4%EC%BB%A4&utm_content=PC_00.%EC%99%80%EB%94%94%EC%A6%88&utm_term=WADIZ&gclid=Cj0KCQjwyZmEBhCpARIsALIzmnJ4L-U5ZdU8R_6y2WpcNpXM6WVwglke6OpUP-fekXUudZSBTXWrlRAaAg0xEALw_wcB'
    webdriver.get(url)
    search = webdriver.find_element_by_css_selector(f'#contents > section.reward-category > div > ul > div > div > li:nth-child('{categories.index(category)}') > a')
    search.click()

    #스크롤 다운
    last_height = webdriver.execute_script("return document.body.scrollHeight")
    body = webdriver.find_element_by_tag_name("body")
    while True:
        last_height = webdriver.execute_script("document.body.scrollHeight")
        scroll_down = 0
        while scroll_down < 10:

            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
            webdriver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")

            time.sleep(0.3)

            scroll_down += 1

        new_height = webdriver.execute_script("document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height  
    
    
    #하나씩 클릭해서 들어가야함 
    page_source = webdriver.page_source
    soup = bs(page_source, 'html.parser')
    contents = soup.findAll('div',class_='ProjectCardList_item__1owJa')
    
    for i in range(1,len(titles)+1): #len(contents)
        btn = webdriver.find_element_by_css_selector(f'div.ProjectCardList_container__3Y14k > div > div:nth-child({i}) > div > div > a > div > span')
        time.sleep(0.5)
        btn.click()
        
        #시계열 데이터
        support_page = webdriver.find_element_by_css_selector('#container > div.reward-nav > ul > li:nth-child(6)')
        time.sleep(1)
        support_page.click()

        button = webdriver.find_element_by_css_selector('#reward-static-supports-list-app > div > div > div > div.ListMoreButton_container__1rOIU > button')
        while button:
            try:
                button.click()
                time.sleep(2)
            except:
                break

        supports = []
        page_source = webdriver.page_source
        soup = bs(page_source, 'html.parser')
        supports_info = soup.findAll('div', class_ = 'RewardSupporterItem_container__1UTDZ')

        for i in supports_info:
            supports.append(i.text.split('\xa0'))
        #     데이터 전처리 필요함

        webdriver.back()
        time.sleep(0.5)
        webdriver.back()

    
    data = supports
    df = pd.DataFrame(data)
    
    return df.to_csv(category+'_tiemseires_df')

    